## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

# choose the optimizer and loss function

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [6]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3331 - accuracy: 0.9041 - val_loss: 0.1600 - val_accuracy: 0.9530 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1381 - accuracy: 0.9599 - val_loss: 0.1112 - val_accuracy: 0.9662 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0982 - accuracy: 0.9708 - val_loss: 0.0906 - val_accuracy: 0.9707 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0743 - accuracy: 0.9772 - val_loss: 0.0758 - val_accuracy: 0.9775 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0600 - accuracy: 0.9819 - val_loss: 0.0581 - val_accuracy: 0.9818 - 2s/epoch - 3ms/step


## Test the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 351ms/step - loss: 0.1160 - accuracy: 0.9659


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.12. Test accuracy: 96.59%
